In [73]:
# author: Jhonathan Romero Fontalvo
#         BK and JW transformations (April 30, 2015)
#         Circuits (August 30, 2015)
#         Commutators (December 2, 2015)
#         Revised version (August 11, 2016)

import time
import numpy as np
import scipy
from scipy.optimize import minimize
import warnings
#import commutators
import scipy.sparse
import scipy.optimize
import scipy.sparse.linalg
from itertools import combinations
from sys import argv
import itertools
import commutators #(Ryan Babbush)
from qutip import *

## Symbolic Bravyi-Kitaev (BK) and Jordan-Wigner (JW) transformations.

The next cells have routines that perform the tasks requires for the Jordan-Wigner and the Bravyi-Kitaev transformation.

In [74]:
def generateBKSets(qubits):
    """ Generate the sets parity (P), update (U), flip (F) and remaining (R) sets
        required for the BK transformation.
        
        References: Seeley et al. JCP, 137, 224109 (2012) 
        
        Returns
        -------
            Lists of P, U, F and R for n qubits. Handles up to 10 qubits.
    """
    dic={0:1,1:2,2:4,3:8,4:16,5:32,6:64,7:128,8:256,9:512,10:1024}
    for n in range(0,11):
        if dic[n]>=qubits:
            power2=n
            break
    beta=1.
    invbeta=1.
    size=2**power2
    pi=np.zeros((size,size))
    for n in range(0,size):
        pi[n,0:n]=1
    I = np.array([[1, 0], [0, 1]])
    for n in range(0,power2):
        l1=2**n
        beta=np.kron(I,beta)
        invbeta=np.kron(I,invbeta)
        for m in range(0,l1):
            beta[2*l1-1,m]=1.0
        invbeta[2*l1-1,l1-1]=1.0
    #print np.linalg.inv(beta)
    parity=np.remainder(np.dot(pi,invbeta),2)
    P={}
    for n in range(0,qubits):
        s=[]
        for m in range(0,n):
            if parity[n,m]==1:
                s.append(m)
        P[n]=s
    R={}
    for n in range(0,qubits):
        s=[]
        for m in range(0,n):
            if parity[n,m]==1:
                s.append(m)
        R[n]=s
    F={}
    for n in range(0,qubits):
        set=[]
        for m in range(0,n):
            if invbeta[n,m]==1:
                set.append(m)
        F[n]=set
    U={}
    for n in range(0,qubits):
        set=[]
        for m in range(n+1,qubits):
            if beta[m,n]==1:
                set.append(m)
        U[n]=set
    for n in range(0,qubits):
        for x in F[n]:
            if x in R[n]:
                R[n].remove(x)
    return beta,P,U,F,R

In [75]:
def chunkIt(seq, num):
    """Create chuncks of a list"""
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

class term:
    """ Class Term: Defines symbolically a product of Pauli Matrices
        The term is saved as a dictionary, with each qubit having a list of terms
    """
    def __init__(self,nqubits,alpha=0.0):
        string={}
        for n in range(0,nqubits):
            string[n]=['I']
        self.c = 1.0 # coefficient of the term
        self.pl = string # dictionary containing n strings, one per each qubit
        self.matrix=[] # matrix form of the term
        self.l=nqubits # length: number of qubits
        self.alpha=alpha #
        
    def reduceTerm(self):
        """
            Reduces a product of Pauli Matrices
        """
        dic1={'ZZ':'I','YY':'I','XX':'I','II':'I','XY':'Z','XZ':'Y','YX':'Z','YZ':'X','ZX':'Y',
              'ZY':'X','IX':'X','IY':'Y','IZ':'Z','ZI':'Z','YI':'Y','XI':'X'}
        dic2={'ZZ':1.0,'YY':1.0,'XX':1.0,'II':1.0,'XY':1.0j,'XZ':-1.0j,'YX':-1.0j,'YZ':1.0j,'ZX':1.0j,
              'ZY':-1.0j,'IX':1.0,'IY':1.0,'IZ':1.0,'ZI':1.0,'YI':1.0,'XI':1.0}
        factor=1.0 
        for x in self.pl:
            t=self.pl[x]
            length=len(t)
            while length>1:
                pair=''.join(t[:2])
                del t[:2]
                t.insert(0,dic1[pair])
                factor=factor*dic2[pair]
                length=len(t)
            self.pl[x]=t
        self.c=factor*self.c
    
    def printTerm(self):
        """
            Routine to print term. N 1.
            
            Returns
            -------
                Coefficient and string representing the term
        """
        vprint=[]
        for x in self.pl:
            vprint += self.pl[x]
            vprint += str(x)
        vprint=''.join(vprint)
        return self.c,vprint
    
    def printTerm2(self):
        """
            Routine to print term. N 2.
            
            Returns
            -------
                Coefficient and string representing the term
        """
        vprint=[]
        for x in self.pl:
            vprint += self.pl[x]
        return self.c,vprint
    
    def printTerm3(self):
        """
            Routine to print term. N 3.
            
            Returns
            -------
                Coefficient and string representing the term
                In addition counds the number of cnot, swap, single qubits and control z
                gates required to implement the evolution of the terms.
                Used to estimate number of gates for the superconducting qubit devices of John Martinis. 
        """
        vprint=[]
        counter=0
        n_swaps=0
        n_x=0; n_y=0
        maxi=0
        mini=self.l
        for x in self.pl:
            if self.pl[x] != ['Y']:
                n_y += 1
            if self.pl[x] != ['X']:
                n_x += 1
            if self.pl[x] != ['I']:
                counter=counter+1
                vprint += self.pl[x]
                if x>maxi:
                    maxi=x
                if x<mini:
                    mini=x
                vprint += str(x)
            elif self.pl[x] == ['I']:
                vprint += [' ']
        for n in range(mini,maxi+1):
            if self.pl[n] == ['I']:
                n_swaps += 2
        vprint=''.join(vprint)
        n_cnots=2*(maxi-mini)-n_swaps
        sqcount=n_x*2+n_y*2+2*n_cnots+6*n_swaps
        czcount=1*(n_cnots)+3*n_swaps
        return self.c,vprint,8-counter,n,sqcount,czcount,n_swaps,n_cnots
    
    def printTerm4(self):
        """
            Routine to print term. N 4.
            
            Returns
            -------
                Coefficient, string representing the term in latex notation and number of 
                Pauli matrices in the string
        """
        vprint=[]
        counter=0
        for x in self.pl:
            if self.pl[x] != ['I']:
                counter=counter+1
                vprint += '\sigma_'
                cosa=self.pl[x][0]
                vprint += self.pl[x]
                vprint += '^'
                vprint += str(x)
        vprint=''.join(vprint)
        return self.c,vprint,counter

    def getMatrix(self):       
        """
            Routine to print term. N 4.
            
            Returns
            -------
                A qutip quantum object (QUTIP) that corresponds to the Pauli term
        """
        pauli={'I':identity(2),'X':sigmax(),'Y':sigmay(),'Z':sigmaz()}
        counter=-1
        l=[]
        for t in self.pl:
            counter += 1
            l.append(pauli[self.pl[t][0]])
        m = tensor(l)
        self.matrix=self.c*m
        return self.c*m
    
def simplifyTerms(products):
    """        
    Input: products, a list of terms with the same size (number of qubits), that might have repeated terms.
    
    Returns
    -------
        A simplified list of terms, without repetitions.    
        
    Note
    -------
        The routine also discards terms with small coefficients.
        THIS ROUTINE IS PRETTY SLOW, WE MUST MAKE IT MORE EFFICIENT
        CHECK FOR ALGORITHMS TO COMPARE LIST ELEMENTS
    """
    error=10**(-12) # threshold to discard terms
    ss=[]
    products2=[]
    indexes=range(0,len(products))
    counter=0
    removed=[]
    for p in indexes:
        if (p in removed)==False:
            s=[]
            s += [p]; removed += [p]; counter +=1
            value,string=products[p].printTerm()
            for q in indexes:
                if (q in removed)==False:
                    value2,string2=products[q].printTerm()
                    if string2==string:
                        s += [q]; removed += [q]; counter +=1
            ss += [s]
        if counter == len(products):
            break
    # calculating sums and eliminating repeated terms
    for x in ss:
        value=0.0
        for y in x:
            value=value+products[y].c
        if abs(value)>error:
            products[x[0]].c=value
            products2.append(products[x[0]])
    del products
    return products2

def returnBKform(operator,P,U,F,R):
    """        
    Input: operator of the form [i,j,-k,-l] that represents a^{/dag}_i a^{/dag}_j a_k a_l. Any length.
            P, U, F and R sets of the corresponding size.

    Returns
    -------
        The representation of the operator in the BK form
    
    """
    # P, U and F must have the same number of elements = nqubits
    l=len(operator)
    size=2*l
    nqubits=len(P)
    BKform=[]
    counter=-1
    for a in operator:
        counter += 1
        s1=term(nqubits)
        s2=term(nqubits)
        j=abs(a)-1
        # case j is even
        if j%2==0:
            s1.pl[j] += 'X'
            #print U[j], P[j], R[j]
            for k in U[j]:
                s1.pl[k] += 'X'
                #print k
            for k in P[j]:
                #print k
                s1.pl[k] += 'Z'
            s2.pl[j] += 'Y'
            for k in U[j]:
                s2.pl[k] += 'X'
                #print k
            for k in P[j]:
                s2.pl[k] += 'Z'
                #print k
            # annihilator
            if a<0:
                s1.c=0.5
                s2.c=0.5j
            # creator
            elif a>0:
                s1.c=0.5
                s2.c=-0.5j              
            BKform += [s1]
            BKform += [s2]
            s1.reduceTerm()
            s2.reduceTerm()
        # case j is odd
        elif j%2==1:
            s1.pl[j] += 'X'
            #print U[j], P[j], R[j]
            for k in U[j]:
                s1.pl[k] += 'X'
                #print k
            for k in P[j]:
                s1.pl[k] += 'Z'
                #print k
            s2.pl[j] += 'Y'
            for k in U[j]:
                s2.pl[k] += 'X'
                #print k
            for k in R[j]:
                s2.pl[k] += 'Z'
                #print k
            # annihilator
            if a<0:
                s1.c=0.5
                s2.c=0.5j
            # creator
            elif a>0:
                s1.c=0.5
                s2.c=-0.5j 
            s1.reduceTerm()
            s2.reduceTerm()
            BKform += [s1]
            BKform += [s2]
        #print s1.printTerm()
        #print s2.printTerm()
        del s1; del s2
    products=[]
    iterables = chunkIt(range(size), size/2)
    counter = 0
    for element in itertools.product(*iterables):
        counter += 1
        s=term(nqubits)
        for t in element:
            for k in range(0,nqubits):
                s.pl[k] += BKform[t].pl[k]
            s.c=s.c*BKform[t].c
        s.reduceTerm()
        products += [s]
        del s
    # finding terms that are equivalent
    products2=simplifyTerms(products)
    return products2

def returnJWform(operator,nqubits):
    """        
    Input: operator of the form [i,j,-k,-l] that represents a^{/dag}_i a^{/dag}_j a_k a_l. Any length.
            Number of qubits. 

    Returns
    -------
        The representation of the operator in the JW form
    
    """
    l=len(operator)
    size=2*l
    BKform=[]
    counter=-1
    for a in operator:
        counter += 1
        s1=term(nqubits) 
        s2=term(nqubits)
        j=abs(a)-1
        # case j is even
        s1.pl[j] += 'X'
        #print U[j], P[j], R[j]
        for k in range(0,j):
            s1.pl[k] += 'Z'
        s2.pl[j] += 'Y'
        for k in range(0,j):
            s2.pl[k] += 'Z'
        # annihilator
        if a<0:
            s1.c=0.5
            s2.c=0.5j
        # creator
        elif a>0:
            s1.c=0.5
            s2.c=-0.5j 
        s1.reduceTerm()
        s2.reduceTerm()
        BKform += [s1]
        BKform += [s2]
        #print s1.printTerm()
        #print s2.printTerm()
        del s1; del s2
    products=[]
    iterables = chunkIt(range(size), size/2)
    counter = 0
    for element in itertools.product(*iterables):
        counter += 1
        s=term(nqubits)
        for t in element:
            for k in range(0,nqubits):
                s.pl[k] += BKform[t].pl[k]
            s.c=s.c*BKform[t].c
        s.reduceTerm()
        products += [s]
        del s
    # finding terms that are equivalent
    products2=simplifyTerms(products)
    return products2    



# Generating Hamiltonians

Functions that transform a Hamiltonian in second quantization to sum of strings of Pauli Matrices. 

In [76]:
def getBKHamiltonian(coefficients,terms,P,U,F,R):
    """        
    Input: A second quantized hamiltonian represented as a list of coeffficients 
            and terms in second quantization. P, U, F and R sets
            for a given number of qubits.
    
    The notation for terms in second quantization is: [i,j,-k,-l] 
    that represents a^{/dag}_i a^{/dag}_j a_k a_l. Any length.

    Returns
    -------
        The Hamiltonian in BK form
    
    """
    products=[]
    conteo=0
    for c,t in zip(coefficients,terms):    
        #print c,t
        cosa = returnBKform(t,P,U,F,R)
        conteo=conteo+len(cosa)
        for element in cosa:
            element.c=c*element.c
            #print element.printTerm()
        products.extend(cosa)
    # finding terms that are equivalent
    products2=simplifyTerms(products)
    return products2  

def getJWHamiltonian(coefficients, terms,nqubits):
    """        
    Input: A second quantized hamiltonian represented as a list of coeffficients 
            and terms in second quantization. Number of qubits. 
    
    The notation for terms in second quantization is: [i,j,-k,-l] 
    that represents a^{/dag}_i a^{/dag}_j a_k a_l. Any length.

    Returns
    -------
        The Hamiltonian in JW form
    
    """
    error=10**(-12)
    #coefficients, terms = commutators.GetHamiltonianTerms(molecule, basis, add_conjugates=True)
    products=[]
    conteo=0
    for c,t in zip(coefficients,terms):    
        #print c,t
        cosa = returnJWform(t,nqubits)
        conteo=conteo+len(cosa)
        for element in cosa:
            element.c=c*element.c
            #print element.printTerm()
        products.extend(cosa)
    # finding terms that are equivalent
    products2=simplifyTerms(products)
    return products2

In [77]:
def reduceHamiltonian(hamiltonian):
    """        
    Input: Hamiltonian with repeated terms. 

    Returns
    -------
        Hamiltonian without repeated terms.
        
    Note
    -------
        THIS ROUTINE NEEDS ACCELERATION
    
    """    
    check_term={}
    result=True
    new_nqubits=0
    for n in range(0,hamiltonian[0].l):
        counter=0
        for x in hamiltonian:
            if x.pl[n]==['I'] or x.pl[n]==['Z']:
                counter+=1
        if counter==len(hamiltonian):
            check_term[n]=False
        else:
            check_term[n]=True
        result=result*check_term[n]
        new_nqubits += check_term[n]
    products=[]
    print (check_term)
    if result==0:
        for x in hamiltonian:
            y=term(int(new_nqubits))
            counter=-1
            y.c=x.c
            for n in range(0,len(check_term)):
                if check_term[n]==True:
                    counter += 1
                    y.pl[counter]=x.pl[n]
            products.append(y)
        products2=simplifyTerms(products)
        return products2
    else:
        return hamiltonian

In [91]:
def checkCommutation(term1,term2):
    """        
    Input: 2 TERMS

    Returns
    -------
        Logical value. False or TrueHamiltonian without repeated terms.
        
    Note
    -------
        THIS ROUTINE NEEDS ACCELERATION
    
    """    
    nqubits1=max(term1.l,term2.l)
    nqubits2=min(term1.l,term2.l)
    product1=term(nqubits1)
    product2=term(nqubits1)
    for x in range(0,nqubits2):
        product1.pl[x] += term1.pl[x]
        product1.pl[x] += term2.pl[x]
        product2.pl[x] += term2.pl[x]
        product2.pl[x] += term1.pl[x]
    product1.reduceTerm(); product2.reduceTerm();
    sameTerm=True
    for x in range(0,nqubits2):
        if product1.pl[x]!=product2.pl[x]:
            sameTerm=False
            break
    commute=False
    if sameTerm==True:
        if product1.c==product2.c:
            commute=True
    return commute

def getHam_JW(molecule,basis,path):
    """        
    Gets a hamiltonian from file. Uses commutators.py

    Returns
    -------
        Hamiltonian in JW form
    """  
    fci_energy, repulsion, coefficients, terms = commutators.GetHamiltonianTerms(molecule, basis, add_conjugates=True, path=path)
    N = commutators.OrbitalCount(terms)
    hamiltonian=getJWHamiltonian(coefficients,terms,N)
    return hamiltonian

def getHam_BK(molecule,basis,path):
    """        
    Gets a hamiltonian from file. Uses commutators.py

    Returns
    -------
        Hamiltonian in BK form
    """  
    fci_energy, repulsion, coefficients, terms = commutators.GetHamiltonianTerms(molecule, basis, add_conjugates=True, path=path)
    N = commutators.OrbitalCount(terms)
    hamiltonian=getBKHamiltonian(coefficients,terms,N)
    return hamiltonian

def getHamiltonianObject(ham):
    """        
    Gets a qutip quantum object from a Hamiltonian consisting of a list of objects of type term. 
    
    Returns
    -------
        Qobj
    """  
    qobject = ham[0].getMatrix()
    for n in range(1,len(ham)):
        qobject = qobject + ham[n].getMatrix()
    return qobject

## Example: getting the representation of the Unitary Coupled Cluster operators (JW and BK representations)

This operators are excitation operators minus their conjugates. For example: $a_i^{\dagger}a_j^{\dagger}a_ka_l-a_l^{\dagger}a_k^{\dagger}a_ka_i$ (double excitation). The double excitation is represented as: [i,j,-k,-l].

In [79]:
def diffJW(example1,nqubits):
    """
    Returns
    -------
        JW form of an antihermitian UCC operator
    """
    example2=example1[::-1]
    example2 = [x * -1 for x in example2]
    expression=getJWHamiltonian([1.0,-1.0],[example1,example2],nqubits)
    return expression

def diffBK(example1,P,U,F,R):
    """
    Returns
    -------
        BK form of an antihermitian UCC operator
    """
    example2=example1[::-1]
    example2 = [x * -1 for x in example2]
    expression=getBKHamiltonian([1.0,-1.0],[example1,example2],P,U,F,R)
    return expression

In [82]:
b,P,U,F,R=generateBKSets(8)
# Example for $a_8^{\dagger}a_7^{\dagger}a_6a_5-a_5^{\dagger}a_6^{\dagger}7_ka_8$ for a system with 8 qubits.
e=diffBK([4,3,-2,-1],P,U,F,R)
counter=0
for x in e:
    counter += 1
    c,t=x.printTerm()
    print(counter,c, t)


(1, 0.125j, 'X0I1Y2I3I4I5I6I7')
(2, 0.125j, 'X0Z1Y2I3I4I5I6I7')
(3, -0.125j, 'Y0I1X2I3I4I5I6I7')
(4, -0.125j, 'Y0Z1X2I3I4I5I6I7')
(5, -0.125j, 'Y0Z1X2Z3I4I5I6I7')
(6, -0.125j, 'Y0I1X2Z3I4I5I6I7')
(7, 0.125j, 'X0Z1Y2Z3I4I5I6I7')
(8, 0.125j, 'X0I1Y2Z3I4I5I6I7')


In [83]:
x=diffJW([4,3,-2,-1],8)
for y in x:
    print
    for z in x:
        print(y.printTerm4(), z.printTerm4())
        print(checkCommutation(z,y))


((0.125j, '\\sigma_X^0\\sigma_X^1\\sigma_Y^2\\sigma_X^3', 4), (0.125j, '\\sigma_X^0\\sigma_X^1\\sigma_Y^2\\sigma_X^3', 4))
True
((0.125j, '\\sigma_X^0\\sigma_X^1\\sigma_Y^2\\sigma_X^3', 4), (-0.125j, '\\sigma_Y^0\\sigma_Y^1\\sigma_Y^2\\sigma_X^3', 4))
True
((0.125j, '\\sigma_X^0\\sigma_X^1\\sigma_Y^2\\sigma_X^3', 4), (-0.125j, '\\sigma_Y^0\\sigma_X^1\\sigma_X^2\\sigma_X^3', 4))
True
((0.125j, '\\sigma_X^0\\sigma_X^1\\sigma_Y^2\\sigma_X^3', 4), (-0.125j, '\\sigma_X^0\\sigma_Y^1\\sigma_X^2\\sigma_X^3', 4))
True
((0.125j, '\\sigma_X^0\\sigma_X^1\\sigma_Y^2\\sigma_X^3', 4), (0.125j, '\\sigma_Y^0\\sigma_X^1\\sigma_Y^2\\sigma_Y^3', 4))
True
((0.125j, '\\sigma_X^0\\sigma_X^1\\sigma_Y^2\\sigma_X^3', 4), (0.125j, '\\sigma_X^0\\sigma_Y^1\\sigma_Y^2\\sigma_Y^3', 4))
True
((0.125j, '\\sigma_X^0\\sigma_X^1\\sigma_Y^2\\sigma_X^3', 4), (0.125j, '\\sigma_X^0\\sigma_X^1\\sigma_X^2\\sigma_Y^3', 4))
True
((0.125j, '\\sigma_X^0\\sigma_X^1\\sigma_Y^2\\sigma_X^3', 4), (-0.125j, '\\sigma_Y^0\\sigma_Y^1\\sig

### Getting all the single and double operators for a system with 8 molecular orbitals in 8 qubits.

In [84]:
def createExcitations(nocc,total,N):
    """
        Return 
        ------
            Create excitation operators up to order N for a given number of occupied orbitals (nocc)
            and total orbitals (total)
    """
    occ=range(1,nocc+1)
    vir=range(nocc+1,total+1)
    operators=[]
    for n in range(1,N+1):
        for cosa1 in itertools.combinations(occ,n):
            for cosa2 in itertools.combinations(vir,n):
                cosita=[]
                cosita.extend(cosa2[::-1])
                cosita.extend([x * -1 for x in cosa1[::-1]])
                operators.append(cosita)
    return operators

In [85]:
x=createExcitations(2,6,2)
print x

[[3, -1], [4, -1], [5, -1], [6, -1], [3, -2], [4, -2], [5, -2], [6, -2], [4, 3, -2, -1], [5, 3, -2, -1], [6, 3, -2, -1], [5, 4, -2, -1], [6, 4, -2, -1], [6, 5, -2, -1]]


### Here we also count the number of SWAPS and CNOTs required to simulate the first term in the sum representing every excitation operator (excitation operators are sums of at least 2 products of Pauli matrices in the BK representation :P).

In [86]:
operators=[]
totalCZ=0
totalSQ=0
for y in x:
    op=diffJW(y,6)
    operators += op
    print y
    for cosa in op:
        a,b,c,d,e,f,g,h=cosa.printTerm3()
        print a,b,c,d,e,f,g,h
        print cosa.printTerm()
        totalSQ += e
        totalCZ += f
        print
    print 'total SQ:', totalSQ, 'total CZ:',totalCZ
    print

[3, -1]
0.5j Y0Z1X2    5 2 28 4 0 4
(0.5j, 'Y0Z1X2I3I4I5')

-0.5j X0Z1Y2    5 2 28 4 0 4
(-0.5j, 'X0Z1Y2I3I4I5')

total SQ: 56 total CZ: 8

[4, -1]
0.5j Y0Z1Z2X3   4 3 32 6 0 6
(0.5j, 'Y0Z1Z2X3I4I5')

-0.5j X0Z1Z2Y3   4 3 32 6 0 6
(-0.5j, 'X0Z1Z2Y3I4I5')

total SQ: 120 total CZ: 20

[5, -1]
0.5j Y0Z1Z2Z3X4  3 4 36 8 0 8
(0.5j, 'Y0Z1Z2Z3X4I5')

-0.5j X0Z1Z2Z3Y4  3 4 36 8 0 8
(-0.5j, 'X0Z1Z2Z3Y4I5')

total SQ: 192 total CZ: 36

[6, -1]
0.5j Y0Z1Z2Z3Z4X5 2 5 40 10 0 10
(0.5j, 'Y0Z1Z2Z3Z4X5')

-0.5j X0Z1Z2Z3Z4Y5 2 5 40 10 0 10
(-0.5j, 'X0Z1Z2Z3Z4Y5')

total SQ: 272 total CZ: 56

[3, -2]
0.5j  Y1X2    6 2 24 2 0 2
(0.5j, 'I0Y1X2I3I4I5')

-0.5j  X1Y2    6 2 24 2 0 2
(-0.5j, 'I0X1Y2I3I4I5')

total SQ: 320 total CZ: 60

[4, -2]
0.5j  Y1Z2X3   5 3 28 4 0 4
(0.5j, 'I0Y1Z2X3I4I5')

-0.5j  X1Z2Y3   5 3 28 4 0 4
(-0.5j, 'I0X1Z2Y3I4I5')

total SQ: 376 total CZ: 68

[5, -2]
0.5j  Y1Z2Z3X4  4 4 32 6 0 6
(0.5j, 'I0Y1Z2Z3X4I5')

-0.5j  X1Z2Z3Y4  4 4 32 6 0 6
(-0.5j, 'I0X1Z2Z3Y4I5')

total SQ: 440 total 

## Example: reading the hamiltonian of H2. 

In [92]:
molecule='H2'
basis='0.75-STO6G-NMO'
path = '/home/jonathan/exacta/integrals/H2/H2_0.75-STO6G-NMO.int'
hamiltonian=getHam_JW(molecule,basis,path)

counter=0
for x in hamiltonian:
    counter += 1
    c,t=x.printTerm()
    print c,t
print

('Inside commutators: path', '/home/jonathan/exacta/integrals/H2/H2_0.75-STO6G-NMO.int')
('commutators path:', '/home/jonathan/exacta/integrals/H2/H2_0.75-STO6G-NMO.int')
('The FCIenergy from integrals file is:', -1.1457416723)
(-0.82294865478+0j) I0I1I2I3
(0.171768685956+0j) Z0I1I2I3
(0.171768685956+0j) I0Z1I2I3
(-0.217367886122+0j) I0I1Z2I3
(-0.217367886122+0j) I0I1I2Z3
(0.168196618521+0j) Z0Z1I2I3
(0.120145678013+0j) Z0I1Z2I3
(0.16566073966+0j) I0Z1Z2I3
(0.16566073966+0j) Z0I1I2Z3
(0.120145678013+0j) I0Z1I2Z3
(0.174337601245+0j) I0I1Z2Z3
(0.045515061647+0j) Y0X1X2Y3
(-0.045515061647+0j) X0X1Y2Y3
(-0.045515061647+0j) Y0Y1X2X3
(0.045515061647+0j) X0Y1Y2X3



In [93]:
molecule='H2'
basis='0.75-STO6G-NMO'
path = '/home/jonathan/exacta/integrals/various/H2-321G-NMO.int'
hamiltonian=getHam_JW(molecule,basis,path)

counter=0
for x in hamiltonian:
    counter += 1
    c,t=x.printTerm()
    print c,t
print

('Inside commutators: path', '/home/jonathan/exacta/integrals/various/H2-321G-NMO.int')
('commutators path:', '/home/jonathan/exacta/integrals/various/H2-321G-NMO.int')
('The FCIenergy from integrals file is:', -1.1478773771)
(-4.99005300302+0j) I0I1I2I3I4I5I6I7
(1.54071571059+0j) Z0I1I2I3I4I5I6I7
(1.54071571059+0j) I0Z1I2I3I4I5I6I7
(1.18024843983+0j) I0I1Z2I3I4I5I6I7
(1.18024843983+0j) I0I1I2Z3I4I5I6I7
(0.867914832498+0j) I0I1I2I3Z4I5I6I7
(0.867914832498+0j) I0I1I2I3I4Z5I6I7
(0.619945561454+0j) I0I1I2I3I4I5Z6I7
(0.619945561454+0j) I0I1I2I3I4I5I6Z7
(-0.163255262607+0j) Z0Z1I2I3I4I5I6I7
(-0.116392894825+0j) Z0I1Z2I3I4I5I6I7
(-0.159297608443+0j) I0Z1Z2I3I4I5I6I7
(-0.159297608443+0j) Z0I1I2Z3I4I5I6I7
(-0.116392894825+0j) I0Z1I2Z3I4I5I6I7
(-0.166328619574+0j) I0I1Z2Z3I4I5I6I7
(-0.11147314531+0j) Z0I1I2I3Z4I5I6I7
(-0.13976660643+0j) I0Z1I2I3Z4I5I6I7
(-0.119801832265+0j) I0I1Z2I3Z4I5I6I7
(-0.137233420139+0j) I0I1I2Z3Z4I5I6I7
(-0.13976660643+0j) Z0I1I2I3I4Z5I6I7
(-0.11147314531+0j) I0Z1I2I3I4

## Example: Hubbard model

In [88]:
def constructSymbolicHubbard(nsites,t,U):
    """
        Generates the Hamiltonian for a Hubbard model 
        (all sites interact, not quite Hubbard model, but for 2 sites is OK)
        
        Inputs: number of sites (nsites), t and U (parameters of the model)
    """
    allQubits = range(1,2*nsites+1)
    spinUpList = [x for x in allQubits if x % 2 == 0]
    spinDownList = [x for x in allQubits if x % 2 == 1]
    coefficients=[]
    operators=[]
    for combination2 in combinations(spinUpList,2):
        operator=[]
        operator.append(combination2[1])
        operator.append(-1*combination2[0])
        operators.append(operator)
        operators.append([-1*x for x in reversed(operator)])
        coefficients.append(-t)
        coefficients.append(-t)
    for combination2 in combinations(spinDownList,2):
        operator=[]
        operator.append(combination2[1])
        operator.append(-1*combination2[0])
        operators.append(operator)
        operators.append([-1*x for x in reversed(operator)])
        coefficients.append(-t)
        coefficients.append(-t)
    for n in spinDownList:
        operator=[]
        operator.extend([n, -n, n+1, -n-1])
        operators.append(operator)
        coefficients.append(U)
    return operators, coefficients

In [61]:
# generate a Hamiltonian with 2 sites

ops,coeffs=constructSymbolicHubbard(2,-1,2)
print(ops)

# generate symbolic hamiltonian (ham) and its matrix form (hamo)
ham=getJWHamiltonian(coeffs, self.c*mops, 4)
print("Symbolic Hamiltonian:")
for x in ham:
    print(x.printTerm())

print("Hamiltonian Qobj:")
hamo=getHamiltonianObject(ham)

# get eigenvalues and eigenvectors using qutip

print("Eigenvalues and eigenvectors")
eigenvals,eigenvecs=hamo.eigenstates()
print(eigenvals)
print(eigenvecs[0])

[[4, -2], [2, -4], [3, -1], [1, -3], [1, -1, 2, -2], [3, -3, 4, -4]]
Symbolic Hamiltonian:
((0.5+0j), 'I0X1Z2X3')
((0.5+0j), 'I0Y1Z2Y3')
((0.5+0j), 'X0Z1X2I3')
((0.5+0j), 'Y0Z1Y2I3')
((1+0j), 'I0I1I2I3')
((-0.5+0j), 'I0Z1I2I3')
((-0.5+0j), 'Z0I1I2I3')
((0.5+0j), 'Z0Z1I2I3')
((-0.5+0j), 'I0I1I2Z3')
((-0.5+0j), 'I0I1Z2I3')
((0.5+0j), 'I0I1Z2Z3')
Hamiltonian Qobj:
Eigenvalues and eigenvectors
[-1.23606798 -1.         -1.          0.          0.          0.          0.
  1.          1.          1.          1.          2.          3.          3.
  3.23606798  4.        ]
Quantum object: dims = [[2, 2, 2, 2], [1, 1, 1, 1]], shape = [16, 1], type = ket
Qobj data =
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.37174803]
 [ 0.        ]
 [ 0.        ]
 [ 0.60150096]
 [ 0.        ]
 [ 0.        ]
 [-0.60150096]
 [ 0.        ]
 [ 0.        ]
 [ 0.37174803]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


# From symbolic to matrix representation (testing circuits).

The purpose of this part is to write circuits. We introduce a dictionary of gates we are going to work with.

In [62]:
# Input: a list of characters representing gates
# Output: the matrix representation of the gates
def getMatrix(term):
    occ = scipy.sparse.csr_matrix([[0], [1]], dtype=float)
    vir = scipy.sparse.csr_matrix([[1], [0]], dtype=float)
    I = scipy.sparse.csr_matrix([[1, 0], [0, 1]], dtype=complex)
    X = scipy.sparse.csr_matrix([[0, 1], [1, 0]], dtype=complex)
    Y = scipy.sparse.csr_matrix([[0, -1j], [1j, 0]], dtype=complex)
    Z = scipy.sparse.csr_matrix([[1, 0], [0, -1]], dtype=complex)
    SWAP=scipy.sparse.csr_matrix([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]], dtype=complex) # S
    H = (1/np.sqrt(2))*scipy.sparse.csr_matrix([[1, 1], [1, -1]], dtype=complex) # H
    CNOT=scipy.sparse.csr_matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]], dtype=complex) # C
    RX2=scipy.sparse.linalg.expm(-(np.pi/4)*1j*X) # A
    RY2=scipy.sparse.linalg.expm(-(np.pi/4)*1j*Y) # B
    R_X2=scipy.sparse.linalg.expm((np.pi/4)*1j*X) # a
    R_Y2=scipy.sparse.linalg.expm((np.pi/4)*1j*Y) # b
    CZ=scipy.sparse.csr_matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, -1]], dtype=complex) # E
    
    # Dictionary of gates
    gates={'I':I,'X':X,'Y':Y,'Z':Z,'H':H,'A':RX2,'B':RY2,'a':R_X2,'b':R_Y2,'C':CNOT,'1':occ,'0':vir,'S':SWAP,'E':CZ}
    # gates
    counter=-1
    m=1
    for t in term:
        counter += 1
        m=scipy.sparse.kron(gates[t],m,'csr')
    return m

# get rotations of single qubit operations
def exponentiate(term,alpha):
    cosa=getMatrix(term)
    expop=scipy.sparse.linalg.expm(-1j*(alpha/2)*cosa)
    return expop

# Phase shift gate (CZ(\phi)). This gate is special because is essential for Superconducting Qubit architectures. 
def CZ(phi):
    m=scipy.sparse.csr_matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, np.exp(1j*phi)]], dtype=complex)
    return m

#### Examples:

In [63]:
# Building a CNOT from CZs.
B=getMatrix(['B'])
A=getMatrix(['A'])
print A
print
print A.getH()
print 
print B
print
print B.getH()
print

CZpi=CZ(np.pi)
p1=getMatrix(['B','I'])
p2=getMatrix(['H','I'])
print
CNOT=p1*CZpi*p1.getH()
CNOT2=p2*CZpi*p2.getH()
if np.allclose(CNOT.todense(),CNOT2.todense())==True:
    print 'the same CNOT'
else:
    print 'CNOTs are different'

print CNOT
print
print CNOT2

  (0, 0)	(0.707106781187+0j)
  (0, 1)	-0.707106781187j
  (1, 0)	-0.707106781187j
  (1, 1)	(0.707106781187+0j)

  (0, 0)	(0.707106781187-0j)
  (1, 0)	0.707106781187j
  (0, 1)	0.707106781187j
  (1, 1)	(0.707106781187-0j)

  (0, 0)	(0.707106781187+0j)
  (0, 1)	(-0.707106781187+0j)
  (1, 0)	(0.707106781187+0j)
  (1, 1)	(0.707106781187+0j)

  (0, 0)	(0.707106781187-0j)
  (1, 0)	(-0.707106781187-0j)
  (0, 1)	(0.707106781187-0j)
  (1, 1)	(0.707106781187-0j)


the same CNOT
  (0, 1)	(1.11022302463e-16+0j)
  (0, 0)	(1+0j)
  (1, 1)	(1+0j)
  (1, 0)	(1.11022302463e-16+0j)
  (2, 3)	(1-6.12323399574e-17j)
  (2, 2)	(2.22044604925e-16+6.12323399574e-17j)
  (3, 3)	6.12323399574e-17j
  (3, 2)	(1-6.12323399574e-17j)

  (0, 0)	(1+0j)
  (1, 1)	(1+0j)
  (2, 3)	(1-6.12323399574e-17j)
  (2, 2)	6.12323399574e-17j
  (3, 3)	6.12323399574e-17j
  (3, 2)	(1-6.12323399574e-17j)


/usr/lib/python2.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:339: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [64]:
# Building exponentiation of products of Pauli matrices from CNOTs
# ZZ, XX and YY

teta=1.2
circuit1=exponentiate(['Z','Z'],teta)
circuit2=exponentiate(['X','X'],teta)
circuit3=exponentiate(['Y','Y'],teta)
# James circuit
p1=getMatrix(['C'])
p2=scipy.sparse.kron(getMatrix(['I']),exponentiate(['Z'],teta))
p22=scipy.sparse.kron(getMatrix(['I']),exponentiate(['Z'],teta))
p3=getMatrix(['C'])
james1=p3*p2*p1
#james2=exponentiate(['H','H'],-np.pi)*p3*p2*p1*exponentiate(['H','H'],np.pi)
U=getMatrix(['B','B'])
james2=U*p3*p22*p1*U.getH()
U=getMatrix(['A','A'])
james3=U*p3*p22*p1*U.getH()

print 

if np.allclose(james3.todense(),circuit3.todense())==True:
    print 'the same'
else:
    print 'they are different'

if np.allclose(james2.todense(),circuit2.todense())==True:
    print 'the same'
else:
    print 'they are different'

if np.allclose(james1.todense(),circuit1.todense())==True:
    print 'the same'
else:
    print 'they are different'


the same
the same
the same


In [65]:
# Another example with more qubits
teta=0.4
circuit1=exponentiate(['Z','Z','Z','Z','Z'],teta)
circuit2=exponentiate(['X','Y','X','Y','X'],teta)
# James circuit
p1=getMatrix(['C','I','I','I'])*getMatrix(['I','C','I','I'])*getMatrix(['I','I','C','I'])*getMatrix(['I','I','I','C'])
p2=scipy.sparse.kron(getMatrix(['I','I','I','I']),exponentiate(['Z'],teta))
p3=getMatrix(['I','I','I','C'])*getMatrix(['I','I','C','I'])*getMatrix(['I','C','I','I'])*getMatrix(['C','I','I','I'])
james1=p3*p2*p1
U=getMatrix(['B','A','B','A','B'])
james2=U*james1*U.getH()

state=getMatrix(['1','1','0','1','0'])

print state.todense().shape
print

if np.allclose(james1.todense(),circuit1.todense())==True:
    print 'the same'
else:
    print 'they are different'

if np.allclose(james2.todense(),circuit2.todense(),rtol=0.0, atol=1e-8)==True:
    print 'the same'
else:
    print 'they are different'
    
#print james2.todense()
#print circuit2.todense()

(32, 1)

the same
the same


In [66]:
# probing that we can exponentiate products including I operations (products with not adyacent qubits) using SWAP gates
# for the spaces.
teta=1.2
circuit1=exponentiate(['Z','I','Z','I','Z'],teta)
circuit2=exponentiate(['X','I','Y','I','X'],teta)
circuit3=exponentiate(['Y','I','X','I','Y'],teta)
# James circuit
p1=getMatrix(['C','I','I','I'])*getMatrix(['I','S','I','I'])*getMatrix(['I','I','C','I'])*getMatrix(['I','I','I','S'])
p2=scipy.sparse.kron(getMatrix(['I','I','I','I']),exponentiate(['Z'],teta))
p3=getMatrix(['I','I','I','S'])*getMatrix(['I','I','C','I'])*getMatrix(['I','S','I','I'])*getMatrix(['C','I','I','I'])
james1=p3*p2*p1
U2=getMatrix(['B','I','a','I','B'])
james2=U2*james1*U2.getH()
U3=getMatrix(['a','I','B','I','a'])
james3=U3*james1*U3.getH()
#james3=U.getH()*james1*U
state=getMatrix(['1','1','0','1','0'])

print state.todense().shape
print

if np.allclose(james1.todense(),circuit1.todense())==True:
    print 'the same'
else:
    print 'they are different'

if np.allclose(james2.todense(),circuit2.todense(),rtol=0.0, atol=1e-07)==True:
    print 'the same'
else:
    print 'they are different'
    
if np.allclose(james3.todense(),circuit3.todense(),rtol=0.0, atol=1e-07)==True:
    print 'the same'
else:
    print 'they are different'

(32, 1)

the same
the same
the same


In [67]:
# input: object of class term representing the operators
# alpha: parameter to be optimized with VQE
# this implements the original circuit for the exponentiation based on CNOTs and SWAPS
def buildExpCircuit(alpha,t):
    # original circuit
    t.getMatrix()
    circuit=scipy.sparse.linalg.expm(-1j*(alpha/2)*t.matrix)
    l=len(t.pl)
    connectivity=np.zeros(l-1)
    maxi=0
    mini=l
    for x in range(0,l-1):
        if (t.pl[x] != ['I']) & (t.pl[x+1] != ['I']) :
            connectivity[x]=1
    for x in range(0,l):            
        if (t.pl[x] != ['I']):
            if x>maxi:
                maxi=x
            if x<mini:
                mini=x
    ## Circuit= part5*part4*part3*part2*part1
    # CNOTs and SWAPs
    part2=1
    part4=1
    for p in range(mini,maxi):
        segment=term(l-1)
        if connectivity[p]==1 or p==(mini):
            segment.pl[p]=['C']
            segment.getMatrix()
            part4=segment.matrix*part4
            part2=part2*segment.matrix
        if connectivity[p]==0 and p!=(mini):
            segment.pl[p]=['S']
            segment.getMatrix()
            part4=segment.matrix*part4
            part2=part2*segment.matrix
    # rotation in the middle
    segment1=term(mini)
    segment2=term(l-mini-1)    
    segment1.getMatrix()
    segment2.getMatrix()
    part3=scipy.sparse.kron(segment2.matrix,scipy.sparse.kron(exponentiate(['Z'],alpha),segment1.matrix))
    # passing to another basis
    segment=term(l)
    for x in t.pl:
        if t.pl[x]==['X']:
            segment.pl[x]=['B']
        if t.pl[x]==['Y']:
            segment.pl[x]=['a']
    segment.getMatrix()
    part5=segment.matrix
    part1=part5.getH()
    #part1=segment.matrix
    #part5=part1.getH()
    unitary=part5*part4*part3*part2*part1
    if np.allclose(unitary.todense(),circuit.todense(),rtol=0.0, atol=1e-07)==True:
        print 'the same'
    else:
        print 'they are different'
    return unitary

In [76]:
print getMatrix(['A'])
print
print getMatrix(['B'])

b,P8,U8,F8,R8=generateBKSets(8)

e=diffBK([8,7,-4,-3],P8,U8,F8,R8)
print e[0].printTerm3()
buildExpCircuit(1.0,e[0])

  (0, 0)	(0.707106781187+0j)
  (0, 1)	-0.707106781187j
  (1, 0)	-0.707106781187j
  (1, 1)	(0.707106781187+0j)

  (0, 0)	(0.707106781187+0j)
  (0, 1)	(-0.707106781187+0j)
  (1, 0)	(0.707106781187+0j)
  (1, 1)	(0.707106781187+0j)
(0.125j, '  X2   Y6 ', 6, 6, 68, 20, 6, 2)


IndexError: tuple index out of range

In [77]:
# input: object of class term representing the operators
# alpha: parameter to be optimized with VQE
# this implements the original circuit for the exponentiation based in terms of CZs only, 
# this is for superconducting qubits
def buildExpCircuit2(alpha,t):
    # original circuit
    t.getMatrix()
    circuit=scipy.sparse.linalg.expm(-1j*(alpha/2)*t.matrix)
    l=len(t.pl)
    connectivity=np.zeros(l-1)
    maxi=0
    mini=l
    for x in range(0,l-1):
        if (t.pl[x] != ['I']) & (t.pl[x+1] != ['I']) :
            connectivity[x]=1
    for x in range(0,l):            
        if (t.pl[x] != ['I']):
            if x>maxi:
                maxi=x
            if x<mini:
                mini=x
    ## Circuit= part5*part4*part3*part2*part1
    # CNOTs and SWAPs
    part2=1
    part4=1
    for p in range(mini,maxi):
        sCZ=term(l-1); sCZ.pl[p]=['E']; sCZ.getMatrix()
        if connectivity[p]==1 or p==(mini):
            syh=term(l); syh.pl[p]=['B']; syh.getMatrix()
            CNOT=syh.matrix*sCZ.matrix*syh.matrix.getH()
            part4=CNOT*part4
            part2=part2*CNOT
        if connectivity[p]==0 and p!=(mini):
            syh=term(l); syh.pl[p]=['B']; syh.getMatrix()
            CNOT=syh.matrix*sCZ.matrix*syh.matrix.getH()
            #syh2=term(l); syh2.pl[p+1]=['B']; syh2.getMatrix()
            #CNOTi=syh2.matrix*sCZ.matrix*syh2.matrix.getH()
            syh=term(l); syh.pl[p]=['B']; syh.getMatrix()
            syh2=term(l); syh2.pl[p]=['b']; syh2.pl[p+1]=['B']; syh2.getMatrix()
            SWAP=syh.matrix*sCZ.matrix*syh2.matrix*sCZ.matrix*syh2.matrix.getH()*sCZ.matrix*syh.matrix.getH()    
            #SWAP=CNOT*CNOTi*CNOT
            part4=SWAP*part4
            part2=part2*SWAP
    # rotation in the middle
    segment1=term(mini)
    segment2=term(l-mini-1)    
    segment1.getMatrix()
    segment2.getMatrix()
    part3=scipy.sparse.kron(segment2.matrix,scipy.sparse.kron(exponentiate(['Z'],alpha),segment1.matrix))
    # passing to another basis
    segment=term(l)
    for x in t.pl:
        if t.pl[x]==['X']:
            segment.pl[x]=['B']
        if t.pl[x]==['Y']:
            segment.pl[x]=['a']
    segment.getMatrix()
    part5=segment.matrix
    part1=part5.getH()
    #part1=segment.matrix
    #part5=part1.getH()
    unitary=part5*part4*part3*part2*part1
    if np.allclose(unitary.todense(),circuit.todense(),rtol=0.0, atol=1e-06)==True:
        print 'the same'
    else:
        print 'they are different'
    return unitary

In [78]:
print getMatrix(['A'])
print
print getMatrix(['B'])
print
print getMatrix(['E'])

b,P4,U4,F4,R4=generateBKSets(4)
e=diffBK([4,3,-2,-1],P4,U4,F4,R4)
print e[0].printTerm3()
print e[1].printTerm3()
print e[2].printTerm3()
print e[3].printTerm3()
print e[4].printTerm3()
print e[5].printTerm3()
print e[6].printTerm3()
print e[7].printTerm3()
buildExpCircuit2(1.0,e[5])

  (0, 0)	(0.707106781187+0j)
  (0, 1)	-0.707106781187j
  (1, 0)	-0.707106781187j
  (1, 1)	(0.707106781187+0j)

  (0, 0)	(0.707106781187+0j)
  (0, 1)	(-0.707106781187+0j)
  (1, 0)	(0.707106781187+0j)
  (1, 1)	(0.707106781187+0j)

  (0, 0)	(1+0j)
  (1, 1)	(1+0j)
  (2, 2)	(1+0j)
  (3, 3)	(-1+0j)
(0.125j, 'X0 Y2 ', 6, 2, 28, 8, 2, 2)
(0.125j, 'X0Z1Y2 ', 5, 2, 20, 4, 0, 4)
(-0.125j, 'Y0 X2 ', 6, 2, 28, 8, 2, 2)
(-0.125j, 'Y0Z1X2 ', 5, 2, 20, 4, 0, 4)
(-0.125j, 'Y0Z1X2Z3', 4, 3, 24, 6, 0, 6)
(-0.125j, 'Y0 X2Z3', 5, 3, 32, 10, 2, 4)
(0.125j, 'X0Z1Y2Z3', 4, 3, 24, 6, 0, 6)
(0.125j, 'X0 Y2Z3', 5, 3, 32, 10, 2, 4)


IndexError: tuple index out of range

In [79]:
l=2
p=0
sCZ=term(l-1); sCZ.pl[p]=['E']; sCZ.getMatrix()
segment=term(l-1)
segment.pl[p]=['C']
segment.getMatrix()
syh=term(l); syh.pl[p]=['B']; syh.getMatrix()
CNOT=syh.matrix*sCZ.matrix*syh.matrix.getH()
print segment.matrix
print
print CNOT


KeyError: 'E'

In [80]:
if np.allclose(CNOT.todense(),segment.matrix.todense(),rtol=0.0, atol=1e-06)==True:
    print 'the same'
else:
    print 'they are different'

NameError: name 'segment' is not defined

In [81]:
# Molmer-Sorensen gate
def U4(teta,phi,size,subset):
  I = scipy.sparse.csr_matrix([[1, 0], [0, 1]], dtype=complex)
  X = scipy.sparse.csr_matrix([[0, 1], [1, 0]], dtype=complex)
  Y = scipy.sparse.csr_matrix([[0, -1j], [1j, 0]], dtype=complex)
  rot=np.cos(phi)*X+np.sin(phi)*Y;
  counter=-1
  U = scipy.sparse.csr_matrix((2**size,2**size),dtype=complex)
  for O1 in subset:
    counter=counter+1
    for O2 in subset[counter+1:]:
      V=1.0
      for n in range(0,size):
        if n==O1:
          V=scipy.sparse.kron(rot,V,'csr');
        elif n==O2:
          V=scipy.sparse.kron(rot,V,'csr');
        else:
          V=scipy.sparse.kron(I,V,'csr');
      U=U+V
  U=scipy.sparse.linalg.expm(-1j*teta*U/2.0)
  return U

# Single rotation
def U1(teta,m,size,rot):
  I = scipy.sparse.csr_matrix([[1, 0], [0, 1]], dtype=complex)
  X = scipy.sparse.csr_matrix([[0, 1], [1, 0]], dtype=complex)
  Y = scipy.sparse.csr_matrix([[0, -1j], [1j, 0]], dtype=complex)
  Z = scipy.sparse.csr_matrix([[1, 0], [0, -1]], dtype=complex)
  pauli={'I':I,'X':X,'Y':Y,'Z':Z}
  U=1.0
  for n in range(0,size):
    if n==m:
      U=scipy.sparse.kron(pauli[rot],U,'csr')
    else:
      U=scipy.sparse.kron(pauli['I'],U,'csr')
  exponent=-1.0j*teta/2.0
  U=scipy.sparse.linalg.expm(exponent*U)
  return U

# state preparation with Molmer Sorensen
def StatePrep(coefficient, subset, size):
  stateprep= U4(np.pi/2,0,size,subset)*\
         U1(coefficient,0,size,'Z')*\
         U4(-np.pi/2,0,size,subset)
  return stateprep

In [82]:
# input: object of class term representing the operators
# alpha: parameter to be optimized with VQE
def buildExpCircuit3(alpha,t):
    # original circuit
    t.getMatrix()
    circuit=scipy.sparse.linalg.expm(-1j*(alpha/2)*t.matrix)
    l=len(t.pl)
    connectivity=np.zeros(l-1)
    maxi=0
    mini=l
    # calculates connectivity
    for x in range(0,l-1):
        if (t.pl[x] != ['I']) & (t.pl[x+1] != ['I']) :
            connectivity[x]=1
    for x in range(0,l):            
        if (t.pl[x] != ['I']):def getExcitationOperators(n_excitations,n_orbitals,n_electrons,occupied_o,frozen_core,active_vir):
  ncore=max(1,frozen_core+1)
  vspace=min(n_orbitals,occupied_o+active_vir+1)
  occ=range(ncore,occupied_o+1)
  vir=range(occupied_o+1,vspace)
  operators=[]
  for n in range(1,n_excitations+1):
    for combination1 in combinations(occ,n):
      for combination2 in combinations(vir,n):
        operator=[]
        operator.extend(combination2[::-1])
        operator.extend([x * -1 for x in combination1[::-1]])
        operators.append(operator)
  return operators

            if x>maxi:
                maxi=x
            if x<mini:b.todense()
                mini=x
    ## Circuit= part5*part4*part3*part2*part1
    # CNOTs and SWAPs
    cnotbackseg=[]
    for p in range(mini,maxi):
        segment=term(l-1)
        if connectivity[p]==1 or p==(mini):
            segment.pl[p]=['C']
            segment.getMatrix()
            cnotbackseg.append(segment)
        if connectivity[p]==0 and p!=(mini):
            segment.pl[p]=['S']
            segment.getMatrix()
            cnotbackseg.append(segment)
    # rotation in the middle

    rseg=term(l,alpha=alpha)
    rseg.pl[mini]=['Rz']
    rseg.getMatrix()
    # passing to another basis
    changebbackseg=term(l)
    for x in t.pl:
        if t.pl[x]==['X']:
            changebbackseg.pl[x]=['B']
        if t.pl[x]==['Y']:
            changebbackseg.pl[x]=['a']
    changebbackseg.getMatrix()
    result=[changebbackseg,cnotbackseg,rseg]
    #unitary=changebback*cnotbackseg*rseg*cnotbackseg.getH()*changebbackseg.getH()
    return result

SyntaxError: invalid syntax (<ipython-input-82-537e19410f61>, line 16)

In [83]:
# Molmer-Sorensen gate                                                                                                                 
def U4(teta,phi,size,subset):
  I = scipy.sparse.csr_matrix([[1, 0], [0, 1]], dtype=complex)
  X = scipy.sparse.csr_matrix([[0, 1], [1, 0]], dtype=complex)
  Y = scipy.sparse.csr_matrix([[0, -1j], [1j, 0]], dtype=complex)
  rot=np.cos(phi)*X+np.sin(phi)*Y;
  counter=-1
  sizeMatrix=2**size
  U = scipy.sparse.csr_matrix((sizeMatrix,sizeMatrix),dtype=complex)
  for O1 in subset:
    counter=counter+1
    for O2 in subset[counter+1:]:
      V=1.0
      for n in range(0,size):
	if n==O1:
          V=scipy.sparse.kron(rot,V,'csr');
        elif n==O2:
          V=scipy.sparse.kron(rot,V,'csr');
        else:
          V=scipy.sparse.kron(I,V,'csr');
      U=U+V
  U=scipy.sparse.linalg.expm(-1j*teta*U/2.0)
  return U

def U1(teta,m,size,rot):
  I = scipy.sparse.csr_matrix([[1, 0], [0, 1]], dtype=complex)
  X = scipy.sparse.csr_matrix([[0, 1], [1, 0]], dtype=complex)
  Y = scipy.sparse.csr_matrix([[0, -1j], [1j, 0]], dtype=complex)
  Z = scipy.sparse.csr_matrix([[1, 0], [0, -1]], dtype=complex)
  pauli={'I':I,'X':X,'Y':Y,'Z':Z}
  U=1.0
  for n in range(0,size):
    if n==m:
      U=scipy.sparse.kron(pauli[rot],U,'csr')
    else:
      U=scipy.sparse.kron(pauli['I'],U,'csr')
  exponent=-1.0j*teta/2.0
  U=scipy.sparse.linalg.expm(exponent*U)
  return U

def stateprepIT1(coefficient, psihf, subset, size):
  stateprep= U4(np.pi/2,0,size,subset)*\
             U1(coefficient,0,size,'Z')*\
             U4(np.pi/2,0,size,subset)
  psi = stateprep*psihf
  return psi

def stateprepIT2(coefficient, psihf, subset, size):
  hi=term(2) 
  hi.pl[0]=['I']
  hi.pl[1]=['H']
  hi.getMatrix()
  stateprep= U1(np.pi,1,size,'X')*U4(np.pi/2,0,size,subset)*\
             U1(coefficient,0,size,'Z')*\
             U4(np.pi/2,0,size,subset)
  psi = stateprep*psihf
  return psi


In [84]:
s11=term(2)
s11.pl[0]=['1']
s11.pl[1]=['1']
s11.getMatrix()
s01=term(2)
s01.pl[0]=['1']
s01.pl[1]=['0']
s01.getMatrix()
YY=term(2)
YY.pl[0]=['Y']
YY.pl[1]=['Y']
YY.getMatrix()
print s11.matrix
print
psi1=stateprepIT1(2.3,s01.matrix,[0,1],2)
print psi1
print
psi2=stateprepIT1(2.3,s11.matrix,[0,1],2)
print psi2
print
psi3=stateprepIT2(2.3,s11.matrix,[0,1],2)
print psi3
print
print psi1.getH()*YY.matrix*psi1
print psi2.getH()*YY.matrix*psi2

KeyError: '1'

In [85]:
g1=term(2)
g1.pl=['X','X']
g2=term(2)
g2.pl=['Z','Z']
M=term(2)
M.pl=['I','Y']

In [86]:
print checkCommutation(g1,M)

print checkCommutation(g2,M)

False
False
